# Methode Exacte : Branch & Bound

Nous proposons un algorithme exacte de resolution du probleme de flow shop de permutation. selon les criteres suivants

## Strategie de Recherche

L'algorithme est parametrable par la strategie de recherche . On propose 2 strategie:

    - Recherche en profondeur : Depth First Search
    - Recherche Meilleur D'abord : Best First Search
selon l'instance les 2 algoritmes peuvent donner des resultats different parfois meme tres performants. 

# Evaluation des noeuds 

In [1]:
import sys
sys.path.append("../") # in order to import fsp
from fsp import branch_and_bound
import matplotlib.pyplot as plt
from utils import Instance, Benchmark
import numpy as np

In [2]:
import plotly.figure_factory as ff
from datetime import datetime
import numpy as np
import plotly.express as px
instance4 = Instance(
    np.array([
        [1,2,3,2],
        [1,4,2,10],
        [3,2,1,5],
        [4,10,3,1],
        [1,5,4,4],
        [2,3,2,6],
        [5,2,1,1],
        [2,3,2,6],
        [5,2,1,1],
    ], dtype=np.int64)
)
results = branch_and_bound.get_results(instance4,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
print(results)
res = instance4.get_chart_data(results)

fig = px.timeline(res['df'], x_start="Start", x_end="Finish", y="Task", color="Resource")
fig.layout.xaxis.update({
        'tickvals' : res['date_ticks'],
        'ticktext' : res['num_tick_labels']
        })
fig.show()

start seq(0, 6, 8, 2, 5, 7, 4, 1, 3)
upper bound 49.0
{'C_max': 42.0, 'order': [0, 2, 1, 4, 5, 8, 3, 7, 6], 'details': {'explored': 229926, 'pruned': 180532, 'leafs': 4, 'time': 9.510990500000162}}


In [3]:
# without heuristique init (uses starting sequence)
results2 = branch_and_bound.get_results(instance4,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
print(results2)
res2 = instance4.get_chart_data(results2)

fig2 = px.timeline(res2['df'], x_start="Start", x_end="Finish", y="Task", color="Resource")
fig2.layout.xaxis.update({
        'tickvals' : res['date_ticks'],
        'ticktext' : res['num_tick_labels']
        })
fig2.show()

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 46.0
{'C_max': 42.0, 'order': [0, 2, 1, 4, 5, 8, 3, 7, 6], 'details': {'explored': 229925, 'pruned': 180532, 'leafs': 3, 'time': 13.578897200000029}}


# Depth-First-Search vs Best-First-Search

In [4]:
jobs = 9
machines = 4
random_mat = np.random.random((jobs,machines)) * 100
randomInstance = Instance(random_mat)
random_mat

array([[35.32448286, 92.83947829, 44.45335275, 11.97868242],
       [25.44713178, 24.89959395, 36.69584561, 85.18217112],
       [ 7.61213287, 57.97127798, 29.87796684,  5.7310157 ],
       [77.54582654, 69.66292543, 69.903242  , 83.88921376],
       [18.52036666,  2.81594966,  6.36583829, 84.80061436],
       [93.75024565, 51.90985804, 26.75846754, 24.93387898],
       [51.38988062, 84.70377803, 91.0061737 , 85.90212284],
       [61.05886779, 60.91772792, 17.43217679,  7.78693919],
       [44.76442532, 50.92888785,  5.92095676, 47.22762331]])

In [5]:
depth_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
depth_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 685.1413929362258


{'C_max': 537.9347261646004,
 'order': [2, 4, 1, 8, 6, 3, 0, 5, 7],
 'details': {'explored': 89458,
  'pruned': 127249,
  'leafs': 15,
  'time': 4.284645800000135}}

In [6]:
best_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.BEST_FIRST_SEARCH,use_heuristique_init=False,log=False)
best_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 685.1413929362258


{'C_max': 537.9347261646004,
 'order': [2, 4, 8, 1, 6, 3, 0, 5, 7],
 'details': {'explored': 69599,
  'pruned': 117634,
  'leafs': 11,
  'time': 6.121075900000051}}

# Impact de la distribution initiale des couts sur les machines

## Instance a couts suivant une loi normale 

In [20]:
jobs = 9
machines = 3
mean_time = 10
std_time = 20
random_mat = np.abs(np.random.normal(loc=mean_time,scale=std_time,size=(jobs,machines)))
randomInstance = Instance(random_mat)
random_mat

array([[ 6.14660478,  4.73821887, 13.84141502],
       [11.90612696, 20.25908262,  4.0995311 ],
       [10.5329873 ,  7.04827564, 38.75436879],
       [49.35888826,  3.85811392, 22.94471792],
       [ 1.28209302, 38.84288877, 26.4862898 ],
       [12.79089309,  9.32053765, 33.82582867],
       [ 2.22365816,  0.57355768, 28.27771009],
       [ 4.12215039, 11.63030875, 26.27321704],
       [33.85946121,  1.66895085, 25.92159443]])

In [21]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(0, 6, 1, 7, 5, 2, 8, 4, 3)
upper bound 231.30949651385404


{'C_max': 223.22188869379303,
 'order': [6, 0, 1, 2, 4, 7, 5, 8, 3],
 'details': {'explored': 407303,
  'pruned': 164961,
  'leafs': 5,
  'time': 13.32788359999995}}

In [22]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 261.43025002663165


{'C_max': 223.22188869379303,
 'order': [6, 0, 1, 2, 4, 7, 5, 8, 3],
 'details': {'explored': 408555,
  'pruned': 163781,
  'leafs': 13,
  'time': 16.015578000000005}}

## Instance a couts suivant une loi uniforme

In [29]:
jobs = 9
machines = 4
low = 10
high = 100
random_mat = np.abs(np.random.uniform(low=low,high=high,size=(jobs,machines)))
randomInstance = Instance(random_mat)
random_mat

array([[28.75948187, 30.08885274, 88.61330802, 12.54000721],
       [65.53163835, 26.94485821, 17.46386911, 42.17436021],
       [55.3295282 , 47.18536725, 83.15180349, 63.55754009],
       [60.88998033, 56.54074946, 51.14439568, 86.64426659],
       [70.42342534, 42.57919888, 26.74380321, 88.04871627],
       [58.73137839, 61.47333225, 65.98010658, 34.38243297],
       [94.09172985, 36.26401766, 57.74166741, 30.95906234],
       [78.51757727, 42.06067473, 98.02980989, 73.30842163],
       [41.05458764, 75.66953873, 50.29092212, 88.16854389]])

In [30]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(1, 0, 6, 5, 4, 2, 8, 3, 7)
upper bound 814.8831249390905


{'C_max': 686.7983997001883,
 'order': [8, 2, 3, 7, 5, 4, 6, 1, 0],
 'details': {'explored': 260601,
  'pruned': 255090,
  'leafs': 23,
  'time': 13.835180400000354}}

In [31]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 813.842189746039


{'C_max': 686.7983997001883,
 'order': [8, 2, 3, 7, 5, 4, 6, 1, 0],
 'details': {'explored': 260601,
  'pruned': 255090,
  'leafs': 23,
  'time': 37.96600939999962}}

## Instance a cout de loi exponentielle

In [44]:
jobs = 9
machines = 4
scale = 10
random_mat = np.random.exponential(scale=scale,size=(jobs,machines))
randomInstance = Instance(random_mat)
random_mat

array([[ 8.67850222,  4.3264421 , 20.29488751, 40.21384655],
       [30.59318083, 12.94590949, 19.17144682,  1.39382758],
       [ 6.69261742,  0.91875524,  0.733197  , 11.09468333],
       [ 7.76209271,  0.91041036,  0.64665727,  7.48669551],
       [ 4.84153552,  4.64264686,  4.49365645, 11.58530051],
       [12.63722871,  2.63389906, 17.63252946, 24.74045613],
       [ 9.07937578,  7.84126981, 14.01722019,  0.2725492 ],
       [17.74234497,  1.74945508, 11.02846333,  0.09056551],
       [16.63812413, 11.74067692,  3.12347859,  8.24455906]])

In [45]:
heur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=True,log=False)
heur_res

start seq(3, 2, 4, 7, 6, 8, 5, 1, 0)
upper bound 198.61259043868293


{'C_max': 127.53348621223064,
 'order': [2, 3, 4, 5, 0, 6, 1, 8, 7],
 'details': {'explored': 124236,
  'pruned': 138278,
  'leafs': 6,
  'time': 7.162785700000313}}

In [47]:
noheur_res = branch_and_bound.get_results(randomInstance,search_strategy=branch_and_bound.DEPTH_FIRST_SEARCH,use_heuristique_init=False,log=False)
noheur_res

start seq(0, 1, 2, 3, 4, 5, 6, 7, 8)
upper bound 138.42231522023357


{'C_max': 127.53348621223064,
 'order': [2, 3, 4, 5, 0, 6, 1, 8, 7],
 'details': {'explored': 124235,
  'pruned': 138278,
  'leafs': 5,
  'time': 7.083648699999685}}